In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
from urllib.parse import urljoin, urlparse
import requests
import time

In [2]:
BASE_URL = "https://manga.in.ua"
DOWNLOAD_FOLDER = Path("data")

MANGAS = {
    "Не роби Цікумене сан": "https://manga.in.ua/chapters/85847-ne-robi-xx-cukimine-san-tom-1-rozdil-1.html",
    "Кузина": "https://manga.in.ua/chapters/81041-kuzina-tom-1-rozdil-1.html",
    "Я запросив вибузову гяру на побачення": "https://manga.in.ua/chapters/95538-ja-sprobuvav-zaprositi-vibuhovu-gjaru-na-pobachennja-tom-1-rozdil-1.html"
}

In [3]:
# --- Налаштування Selenium ---
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1200,800")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(5)

In [4]:
def download_chapter(driver: webdriver.Chrome, out_dir: Path):
    # кнопка "Читати розділ" (без except)
    read_btns = driver.find_elements(
        By.CSS_SELECTOR, 'a[onclick*="LoadMangaImages"]'
    )
    if read_btns:
        read_btns[0].click()
        print("Кнопка 'Читати розділ' натиснута")
    else:
        print("Кнопки 'Читати розділ' немає — контент вже завантажений")

    # чекаємо контейнер з картинками
    comics_div = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "comics"))
    )
    images = WebDriverWait(comics_div, 20).until(
        lambda d: d.find_elements(By.CSS_SELECTOR, "img[data-src]")
    )
    if not images:
        raise RuntimeError("Не знайдено картинок у розділі")

    # завантажуємо
    out_dir.mkdir(parents=True, exist_ok=True)
    for idx, img in enumerate(images, start=1):
        img_url = urljoin(BASE_URL, img.get_attribute("data-src"))
        ext = Path(urlparse(img_url).path).suffix or ".jpg"
        filename = out_dir / f"{idx:04d}{ext}"
        filename.write_bytes(requests.get(img_url).content)

    print(f"Завантажено {len(images)} сторінок у {out_dir}")

In [5]:
for manga_name, start_url in MANGAS.items():
    print(f"\n===== МАНГА: {manga_name} =====")

    driver.get(start_url)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "comics"))
    )

    chapter_i = 1
    while True:
        chapter_url = driver.current_url
        folder_name = chapter_url.split("/")[-1].replace(".html", "")
        folder_name = f"{chapter_i:04d}_{folder_name}"
        chapter_i += 1
        out_folder = DOWNLOAD_FOLDER / manga_name / folder_name

        print(f"\n=== Розділ: {folder_name} ===")

        try:
            download_chapter(driver, out_folder)
        except Exception as e:
            print(f"Помилка розділу: {e}")
            break

        # кнопка "Наступний"
        next_btns = driver.find_elements(By.ID, "fwdTop")
        if not next_btns:
            print("Кнопка 'Наступний' відсутня — кінець манги")
            break

        next_btn = next_btns[0]
        if "disabled" in next_btn.get_attribute("class"):
            print("Останній розділ манги")
            break

        old_url = driver.current_url
        next_btn.click()

        WebDriverWait(driver, 20).until(
            lambda d: d.current_url != old_url
        )

    print(f"Манга '{manga_name}' завершена")


===== МАНГА: Не роби Цікумене сан =====

=== Розділ: 0001_85847-ne-robi-xx-cukimine-san-tom-1-rozdil-1 ===
Кнопка 'Читати розділ' натиснута
Завантажено 19 сторінок у data/Не роби Цікумене сан/0001_85847-ne-robi-xx-cukimine-san-tom-1-rozdil-1

=== Розділ: 0002_85848-ne-robi-xx-cukimine-san-tom-1-rozdil-2 ===
Кнопка 'Читати розділ' натиснута
Завантажено 16 сторінок у data/Не роби Цікумене сан/0002_85848-ne-robi-xx-cukimine-san-tom-1-rozdil-2

=== Розділ: 0003_85849-ne-robi-xx-cukimine-san-tom-1-rozdil-3 ===
Кнопка 'Читати розділ' натиснута
Завантажено 17 сторінок у data/Не роби Цікумене сан/0003_85849-ne-robi-xx-cukimine-san-tom-1-rozdil-3

=== Розділ: 0004_85850-ne-robi-xx-cukimine-san-tom-1-rozdil-4 ===
Кнопка 'Читати розділ' натиснута
Завантажено 14 сторінок у data/Не роби Цікумене сан/0004_85850-ne-robi-xx-cukimine-san-tom-1-rozdil-4

=== Розділ: 0005_85851-ne-robi-xx-cukimine-san-tom-1-rozdil-5 ===
Кнопка 'Читати розділ' натиснута
Завантажено 14 сторінок у data/Не роби Цікумене сан

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.193)
Stacktrace:
0   chromedriver                        0x0000000104997dfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x000000010498fcb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x000000010448ab90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x0000000104464948 chromedriver + 149832
4   chromedriver                        0x00000001044f9b38 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 528760
5   chromedriver                        0x000000010451258c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 629708
6   chromedriver                        0x00000001044c622c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317548
7   chromedriver                        